In [1]:
import pandas as pd
import nltk
nltk.download('stopwords')
import time
notebookstart= time.time()
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

# Models Packages
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn import feature_selection
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
# Gradient Boosting
#import lightgbm as lgb

# Tf-Idf
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from scipy.sparse import hstack, csr_matrix
from nltk.corpus import stopwords
from string import punctuation

#HashingVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

import seaborn as sns
import matplotlib.pyplot as plt


[nltk_data] Downloading package stopwords to /home/zyc62/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('demo10100.csv')

In [3]:
print("\nData Load Stage")
df = pd.read_csv('demo10100.csv',parse_dates = ["activation_date"])
df_index = df.index
print(df_index)

#testing = pd.read_csv('test.csv',parse_dates = ["activation_date"])
#testdex = testing.index
#test_item = testing['item_id']

y = df['deal_probability'].copy()
df.drop("deal_probability",axis=1, inplace=True)

#print('Train shape: {} Rows, {} Columns'.format(*training.shape))
#print('Test shape: {} Rows, {} Columns'.format(*testing.shape))

df["price"] = np.log(df["price"]+0.001)               # filling in NaN's
df["price"].fillna(-999,inplace=True)
df["image_top_1"].fillna(-999,inplace=True)
df['param_1'].fillna('fill',inplace=True)
df['param_2'].fillna('fill',inplace=True)
df['param_3'].fillna('fill',inplace=True)

df["Weekday"] = df['activation_date'].dt.weekday
df["Weekd of Year"] = df['activation_date'].dt.week
df["Day of Month"] = df['activation_date'].dt.day

import nltk
nltk.download('stopwords')

russian_stop = set(stopwords.words('russian','fill'))

df['desc_punctuation_cnt'] = df['description'].apply(lambda x: len("".join(_ for _ in str(x) if _ in punctuation)))
df['desc_upper_case_word_cnt'] = df['description'].apply(lambda x: len([wrd for wrd in str(x).split() if wrd.isupper()]))
df['stopword_count'] = df['description'].apply(lambda x: len([wrd for wrd in str(x).split() if wrd.lower() in russian_stop]))

df.drop(["activation_date","image"],axis=1,inplace=True)



Data Load Stage
RangeIndex(start=0, stop=10100, step=1)
[nltk_data] Downloading package stopwords to /home/zyc62/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
print("\nEncode Variables")
categorical = ["region","city","parent_category_name","category_name","user_type","image_top_1"]
print("Encoding :",categorical)

# Encoder:
lbl = preprocessing.LabelEncoder()
for col in categorical:
    df[col] = lbl.fit_transform(df[col].astype(str))

df['text_feat'] = df.apply(lambda row: ' '.join([
    str(row['param_1']),
    str(row['param_2']),
    str(row['param_3'])]),axis=1) # Group Param Features into a single string

df.drop(["param_1","param_2","param_3"],axis=1,inplace=True)


textfeats = ["description","text_feat", "title"]

for cols in textfeats:
    df[cols] = df[cols].astype(str)
    df[cols] = df[cols].astype(str).fillna('nicapotato') # WHY FILL NANS WITH NICAPOTATO?
    df[cols] = df[cols].str.lower() # Lowercase all text, so that capitalized words dont get treated differently

    df[cols + '_num_chars'] = df[cols].apply(len) # Count number of Characters
    df[cols + '_num_words'] = df[cols].apply(lambda comment: len(comment.split())) # Count number of Words
    df[cols + '_num_unique_words'] = df[cols].apply(lambda comment: len(set(w for w in comment.split())))
    df[cols + '_words_vs_unique'] = df[cols+'_num_unique_words'] / df[cols+'_num_words'] * 100 # Count Unique Words



Encode Variables
Encoding : ['region', 'city', 'parent_category_name', 'category_name', 'user_type', 'image_top_1']


In [5]:
tfidf_para = {
    "stop_words": russian_stop,
    "analyzer": 'word',
    "token_pattern": r'\w{1,}',
    "sublinear_tf": True,
    "dtype": np.float32,
    "norm": 'l2',
    #"min_df":5,
    #"max_df":.9,
    "smooth_idf":False
}


def get_col(col_name): return lambda x: x[col_name]

vectorizer = FeatureUnion([
        ('description',TfidfVectorizer(
            ngram_range=(1, 2),
            max_features=16000,
            **tfidf_para,
            preprocessor=get_col('description'))),
        ('text_feat',CountVectorizer(
            ngram_range=(1, 2),
            max_features=5000,
            preprocessor=get_col('text_feat'))),
        ('title',TfidfVectorizer(
            ngram_range=(1, 2),
            **tfidf_para,
            max_features=5000,
            preprocessor=get_col('title')))
    ])


start_vect=time.time()
vectorizer.fit(df.to_dict('records'))

ready_df = vectorizer.transform(df.to_dict('records'))
print(ready_df.shape[0])

10100


In [6]:
tfvocab = vectorizer.get_feature_names()
print("Vectorization Runtime: %0.2f Minutes"%((time.time() - start_vect)/60))

df.drop(['item_id','description','title','text_feat'],axis=1,inplace=True)

df.drop('user_id',axis=1,inplace=True)

Vectorization Runtime: 0.05 Minutes


In [7]:
with pd.option_context('display.max_rows',50, 'display.max_columns', 50):
    display(df.head())

,Unnamed: 0,region,city,parent_category_name,category_name,price,item_seq_number,user_type,image_top_1,id,dullness,whiteness,average_pixel_width,blurrness,resnet50_score,Weekday,Weekd of Year,Day of Month,desc_punctuation_cnt,desc_upper_case_word_cnt,stopword_count,description_num_chars,description_num_words,description_num_unique_words,description_words_vs_unique,text_feat_num_chars,text_feat_num_words,text_feat_num_unique_words,text_feat_words_vs_unique,title_num_chars,title_num_words,title_num_unique_words,title_words_vs_unique
0,0,15,256,0,40,6.214610,21,1,1311,3314,84.56,0.00,4.33,632.23,0.43,1,13,28,2,0,1,49,7,7,100.000000,17,3,2,66.666667,29,5,5,100.0
1,1,3,102,2,4,9.210340,2,1,667,3002,0.00,0.00,1.48,453.89,0.39,1,12,21,1,0,0,27,3,3,100.000000,48,7,7,100.000000,38,5,5,100.0
2,2,10,241,0,43,7.346011,129,0,1246,1797,0.00,87.59,3.98,1654.58,1.00,5,11,18,2,0,6,128,23,22,95.652174,32,4,4,100.000000,10,2,2,100.0
3,3,22,478,7,33,-999.000000,2,1,228,4791,20.49,44.58,4.38,518.58,0.46,2,11,15,13,0,8,367,45,41,91.111111,16,3,2,66.666667,17,2,2,100.0
4,4,13,354,4,41,7.313221,12,1,1824,8583,1.08,87.29,4.26,383.30,0.47,1,12,21,3,1,2,66,9,9,100.000000,30,5,4,80.000000,18,3,3,100.0


In [8]:
df.drop(['Unnamed: 0','Weekday','Weekd of Year','Day of Month','description_num_chars','title_num_chars','id'],axis=1,inplace=True)

with pd.option_context('display.max_rows',50, 'display.max_columns', 50):
    display(df.head())
print(df.shape)

,region,city,parent_category_name,category_name,price,item_seq_number,user_type,image_top_1,dullness,whiteness,average_pixel_width,blurrness,resnet50_score,desc_punctuation_cnt,desc_upper_case_word_cnt,stopword_count,description_num_words,description_num_unique_words,description_words_vs_unique,text_feat_num_chars,text_feat_num_words,text_feat_num_unique_words,text_feat_words_vs_unique,title_num_words,title_num_unique_words,title_words_vs_unique
0,15,256,0,40,6.214610,21,1,1311,84.56,0.00,4.33,632.23,0.43,2,0,1,7,7,100.000000,17,3,2,66.666667,5,5,100.0
1,3,102,2,4,9.210340,2,1,667,0.00,0.00,1.48,453.89,0.39,1,0,0,3,3,100.000000,48,7,7,100.000000,5,5,100.0
2,10,241,0,43,7.346011,129,0,1246,0.00,87.59,3.98,1654.58,1.00,2,0,6,23,22,95.652174,32,4,4,100.000000,2,2,100.0
3,22,478,7,33,-999.000000,2,1,228,20.49,44.58,4.38,518.58,0.46,13,0,8,45,41,91.111111,16,3,2,66.666667,2,2,100.0
4,13,354,4,41,7.313221,12,1,1824,1.08,87.29,4.26,383.30,0.47,3,1,2,9,9,100.000000,30,5,4,80.000000,3,3,100.0


(10100, 26)


In [9]:
print("Modeling Stage")
#Combine Dense Features with Sparse Text Bag of Words Features
X = hstack([csr_matrix(df.loc[df_index,:].values),ready_df[:df_index.shape[0]]]) # Sparse Matrix


#testing = hstack([csr_matrix(df.loc[testdex,:].values),ready_df[traindex.shape[0]:]])

tfvocab = df.columns.tolist() + tfvocab

x = ready_df[:df_index.shape[0]]

print(x.shape)


for shape in [X]:
    print("{} Rows and {} Cols".format(*shape.shape))

print("Feature Names Length: ",len(tfvocab))
#del df
gc.collect();

Modeling Stage
(10100, 22996)
10100 Rows and 23022 Cols
Feature Names Length:  23022


In [61]:
from sklearn.decomposition import TruncatedSVD
from sklearn import linear_model
from math import sqrt
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler

for i in (400,500,600,700,800,1000):
    svd = TruncatedSVD(n_components= i, n_iter=7, random_state=42).fit(x)
    x_reduced = svd.transform(x)
    print(x_reduced.shape)   
    X1 = hstack([csr_matrix(df.loc[df_index,:].values),x_reduced[:df_index.shape[0]]]) # Sparse Matrix
    print(X1.shape)
#tfvocab = df.columns.tolist() + tfvocab

    X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y, test_size = 100, random_state = 23)


    ss = StandardScaler(with_mean=False)
    X1_train_scaled = ss.fit_transform(X1_train)
    X1_test_scaled = ss.transform(X1_test)



    def plot_ridge():
        ridge_model = linear_model.Ridge(alpha = 5.0).fit(X1_train_scaled,y1_train)
        ridge_predicted = ridge_model.predict(X1_test_scaled)
        ridge_rms = sqrt(mean_squared_error(y1_test, ridge_predicted))
    
        print (ridge_rms)  
    

    plot_ridge()

(10100, 400)
(10100, 426)
0.25201063261886825
(10100, 500)
(10100, 526)
0.24777222486180134
(10100, 600)
(10100, 626)
0.2441487238712897
(10100, 700)
(10100, 726)
0.2386257458670039
(10100, 800)
(10100, 826)
0.24732121244978422
(10100, 1000)
(10100, 1026)
0.2507018392160965


In [11]:
from sklearn.decomposition import TruncatedSVD
from sklearn import linear_model
from math import sqrt
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler

for i in (1200,1400,1600,1800,2000,2500,3000,4000,5000):
    svd = TruncatedSVD(n_components= i, n_iter=7, random_state=42).fit(x)
    x_reduced = svd.transform(x)
    print(x_reduced.shape)   
    X1 = hstack([csr_matrix(df.loc[df_index,:].values),x_reduced[:df_index.shape[0]]]) # Sparse Matrix
    print(X1.shape)
#tfvocab = df.columns.tolist() + tfvocab

    X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y, test_size = 100, random_state = 23)


    ss = StandardScaler(with_mean=False)
    X1_train_scaled = ss.fit_transform(X1_train)
    X1_test_scaled = ss.transform(X1_test)



    def plot_ridge():
        ridge_model = linear_model.Ridge(alpha = 5.0).fit(X1_train_scaled,y1_train)
        ridge_predicted = ridge_model.predict(X1_test_scaled)
        ridge_rms = sqrt(mean_squared_error(y1_test, ridge_predicted))
    
        print (ridge_rms)  
    
    
    plot_ridge()

(10100, 1200)
(10100, 1226)
0.2577141240946108
(10100, 1400)
(10100, 1426)
0.26436129216145415
(10100, 1600)
(10100, 1626)
0.26899949346102237
(10100, 1800)
(10100, 1826)
0.27406335710519486
(10100, 2000)
(10100, 2026)
0.2814122468186412
(10100, 2500)
(10100, 2526)
0.2884359214103392
(10100, 3000)
(10100, 3026)
0.2981659903801435
(10100, 4000)
(10100, 4026)
0.3172618121565679
(10100, 5000)
(10100, 5026)
0.4559833761218537


In [11]:
print(df.loc[df_index,:].shape)

(10100, 26)


In [14]:
x26=df.loc[df_index,:]
X26_train, X26_test, y26_train, y26_test = train_test_split(x26, y, test_size = 100, random_state = 23)
from sklearn.preprocessing import StandardScaler

ss = StandardScaler(with_mean=False)
X26_train_scaled = ss.fit_transform(X26_train)
X26_test_scaled = ss.transform(X26_test)


def plot_ridge():
    ridge_model = linear_model.Ridge(alpha = 5.0).fit(X26_train_scaled,y26_train)
    ridge_predicted = ridge_model.predict(X26_test_scaled)
    ridge_rms = sqrt(mean_squared_error(y26_test, ridge_predicted))
    
    print (ridge_rms)  
plot_ridge()

0.2576796538700181
